In [1]:
# Prepare import of modules from parent directory.
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

"""
Get all relevant observations for one random hour (for testing purposes).
"""

from studies import db

from studies.cycles.lib import query_builder
from preparations import things_provider
from _config import TIMERANGES, CURRENT_IDENTIFIER, OBSERVATIONS_CSV_FILE


tp = things_provider.ThingsProvider()
tp.filter_only_bike_things()
tp.filter_only_primary_signal_datastreams()
primary_signal_datastream_ids_bike = tp.get_datastream_ids()

tp = things_provider.ThingsProvider()
tp.filter_only_bike_things()
tp.filter_only_cycle_second_datastreams()
cycle_second_datastream_ids_bike = tp.get_datastream_ids()

tp = things_provider.ThingsProvider()
tp.filter_only_bike_things()
tp.filter_only_signal_program_datastreams()
program_signal_datastream_ids_bike = tp.get_datastream_ids()

datastream_ids = primary_signal_datastream_ids_bike + cycle_second_datastream_ids_bike + program_signal_datastream_ids_bike

csv_header = "phenomenon_time,result_time,received_time,result,datastream_id,mqtt"
csv = csv_header + "\n"

with open(OBSERVATIONS_CSV_FILE, "w") as f:
    f.write(csv)

db = db.DBClient()
relevant_observations_query_bike = query_builder.get_relevant_observations_for_given_timeranges(datastream_ids, TIMERANGES[CURRENT_IDENTIFIER])
relevant_observations_rows_bike_iterator = db.execute_query(relevant_observations_query_bike)

csv_row_count = 0
csv_rows = ""
for row in relevant_observations_rows_bike_iterator:
    csv_rows += ",".join([str(x) for x in row]) + "\n"
    # Write to file every 100000 rows. This is to avoid memory issues.
    if csv_row_count >= 100000:
        with open(OBSERVATIONS_CSV_FILE, "a") as f:
            f.write(csv_rows)
        csv_rows = ""
        csv_row_count = 0
    csv_row_count += 1

db.close()


Amount of datastreams: 5418
Amount of datastream ids: 5418
Amount of datastreams: 5419
Amount of datastream ids: 5419
Amount of datastreams: 5418
Amount of datastream ids: 5418
